In [1]:
import os

import pandas as pd
import numpy as np

# Matplotlib for visualization
from matplotlib import pyplot as plt
# display plots in the notebook
%matplotlib inline

# Seaborn for easier visualization
import seaborn as sns
### sns.set_style('darkgrid')

# store elements as dictionary keys and their counts as dictionary values
from collections import Counter

# scikit-learn
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import make_pipeline

# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report

# Function for creating model pipelines - sklearn
from sklearn.pipeline import make_pipeline

# Function for creating model pipelines - imblearn
from imblearn.pipeline import make_pipeline as imbl_pipe

# Over-sampling using SMOTE
from imblearn.over_sampling import SMOTE



## Load Analytical Base Table

In [2]:
abt = pd.read_csv("../Resources/Analytical_Base_Table.csv")
abt.head()

,CustomerID,Gender,Senior_Citizen,Partner,Dependents,Tenure,Phone_Service,Multiple_Lines,Internet_Service,Online_Security,...,Device_Protection,Tech_Support,Streaming_TV,Streaming_Movies,Contract,Paperless_Billing,Payment_Method,Monthly_Charges,Total_Charges,Churn
0,7590-VHVEG,0,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,1,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,1


### Separate dataframe into separate object

In [3]:
X = abt.drop(["CustomerID","Churn"], axis=1)

y = abt["Churn"]

# display shapes of X and y
print(X.shape, y.shape)

(6499, 19) (6499,)


In [4]:
# List numerical features
num_columns = X.select_dtypes(include='number').columns.tolist()
num_columns

['Gender', 'Senior_Citizen', 'Tenure', 'Monthly_Charges', 'Total_Charges']

In [5]:
# List categorical features
cat_columns = X.select_dtypes(include='object').columns.tolist()
cat_columns

['Partner',
 'Dependents',
 'Phone_Service',
 'Multiple_Lines',
 'Internet_Service',
 'Online_Security',
 'Online_Backup',
 'Device_Protection',
 'Tech_Support',
 'Streaming_TV',
 'Streaming_Movies',
 'Contract',
 'Paperless_Billing',
 'Payment_Method']

In [6]:
# distribution of churn
col1 = y.value_counts()
col2 = y.value_counts(normalize=True)*100
pd.DataFrame(data={'Target':[0,1],'Churn':col1,'Churn %':col2.round(2)})

,Target,Churn,Churn %
0,0,4784,73.61
1,1,1715,26.39


## Create a Train Test Split

In [7]:
random_state = 10

# Split X and y into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.3,
                                                    random_state=random_state,
                                                    stratify=abt.Churn)

# Print number of observations in X_train, X_test, y_train, and y_test
print(len(X_train), len(X_test), len(y_train), len(y_test))

4549 1950 4549 1950


In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4549 entries, 736 to 5374
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Gender             4549 non-null   int64  
 1   Senior_Citizen     4549 non-null   int64  
 2   Partner            4549 non-null   object 
 3   Dependents         4549 non-null   object 
 4   Tenure             4549 non-null   int64  
 5   Phone_Service      4549 non-null   object 
 6   Multiple_Lines     4549 non-null   object 
 7   Internet_Service   4549 non-null   object 
 8   Online_Security    4549 non-null   object 
 9   Online_Backup      4549 non-null   object 
 10  Device_Protection  4549 non-null   object 
 11  Tech_Support       4549 non-null   object 
 12  Streaming_TV       4549 non-null   object 
 13  Streaming_Movies   4549 non-null   object 
 14  Contract           4549 non-null   object 
 15  Paperless_Billing  4549 non-null   object 
 16  Payment_Method     454

## Pre-processing Pipeline

### Scale numerical data and encode categorical data
Construct a pre-processing pipeline from the given transformers: MinMaxScaler and OneHotEncoder

Create lists of indexes from the list of column names

Need to be numeric not string to specify columns name in column transformer

In [9]:
num_features = [] 

for i in num_columns:
    location = X.columns.get_loc(i)
    num_features.append(location)
print(num_features)  

[0, 1, 4, 17, 18]


In [10]:
cat_features = []

for i in cat_columns:
    location = X.columns.get_loc(i)
    cat_features.append(location)
print(cat_features)  

[2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]


In [11]:
# Define column transformer
# Need to be numeric not string to specify columns name 
preprocess = make_column_transformer(
    (MinMaxScaler(), num_features),
    (OneHotEncoder(sparse=False), cat_features)
)
preprocess

ColumnTransformer(transformers=[('minmaxscaler', MinMaxScaler(),
                                 [0, 1, 4, 17, 18]),
                                ('onehotencoder', OneHotEncoder(sparse=False),
                                 [2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,
                                  16])])

## Build Model Pipeline with SMOTE

* We are going to use the Pipeline from the imblearn package in place of scikit-learn Pipeline.

* It takes care automatically to re-sample when called fit() on the pipeline, and does not re-sample test data (when called transform() or predict()).

In [15]:
# Import XGBoost
!pip install xgboost
import xgboost as xgb

# Define model with pipeline
model = imbl_pipe(preprocess,
                  SMOTE(sampling_strategy='auto', random_state=random_state),
                  xgb.XGBClassifier(random_state=random_state))

model

     ---------------------------------------- 89.1/89.1 MB 8.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('minmaxscaler',
                                                  MinMaxScaler(),
                                                  [0, 1, 4, 17, 18]),
                                                 ('onehotencoder',
                                                  OneHotEncoder(sparse=False),
                                                  [2, 3, 5, 6, 7, 8, 9, 10, 11,
                                                   12, 13, 14, 15, 16])])),
                ('smote', SMOTE(random_state=10)),
                ('xgbclassifier',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=N...
                               feature_types=None, gamma=None, gpu_id=None,
                               grow_policy=None, importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, n_estimators=100,
                               n_jobs=None, num_parallel_tree=None,
                               predictor=None, random_state=10, ...))])

In [ ]:
xgb.XGBClassifier().get_params()

In [ ]:
# Create the GridSearchCV model
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV

param_grid = {'xgbclassifier__gamma': [0.5, 0.8, 1],
              'xgbclassifier__max_depth': [3, 4, 5, 6],
              'xgbclassifier__n_estimators': [50, 100, 200]
        }
grid = GridSearchCV(model, param_grid, verbose=3, cv= 5, n_jobs=4, scoring='f1_macro')


In [ ]:
# Train the model with GridSearch
grid.fit(X_train, y_train)


In [ ]:
print(grid.best_params_)

In [ ]:
 # List the best score
print(grid.best_score_)

In [ ]:
print(f"Training Data Score: {grid.score(X_train, y_train)}")
print(f"Testing Data Score: {grid.score(X_test, y_test)}")

In [ ]:
# Make predictions with the hypertuned model
pred = grid.predict(X_test)
pred

In [ ]:
# Classification metrics
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, pred)
print(cm)

In [ ]:
cm = np.around(cm / cm.sum(axis=1)[:, np.newaxis], 2)
print(cm)

In [ ]:
print(classification_report(y_test, pred))

In [ ]:
print(f"Predicted classes: \t{list(pred[:10])}")
print(f"Actual Labels: \t\t{list(y_test[:10])}")

## Save the Model

In [ ]:
import joblib

# We are saving our grid model
filename = '../models/challa_XGBoost_model.sav'
joblib.dump(grid, filename)

## Loading the Model

In [ ]:
# load the model
xgb_model = joblib.load(filename)
print(xgb_model.score(X_test, y_test))

### Predict class for new data

In [ ]:
# Let's use the first X_test record as new data
X_test[:1]

In [ ]:
pred_new = xgb_model.predict(X_test[:1])

In [ ]:
print(f"Predicted classes: \t{pred_new}")
print(f"Actual Labels: \t\t{list(y_test[:1])}")

#### Original dataframe to numpy array

In [ ]:
X_test[:1].to_numpy()

In [ ]:
pred_new1 = xgb_model.predict(X_test[:1].to_numpy())
pred_new1

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# defining a function to compute different metrics to check performance of a classification model built using sklearn
def model_performance_classification_sklearn(model, predictors, target):
    """
    Function to compute different metrics to check classification model performance

    model: classifier
    predictors: independent variables
    target: dependent variable
    """

    # predicting using the independent variables
    pred = model.predict(predictors)

    acc = accuracy_score(target, pred)  # to compute Accuracy
    recall = recall_score(target, pred)  # to compute Recall
    precision = precision_score(target, pred)  # to compute Precision
    f1 = f1_score(target, pred)  # to compute F1-score

    # creating a dataframe of metrics
    df_perf = pd.DataFrame(
        {"Accuracy": acc, "Recall": recall, "Precision": precision, "F1": f1,},
        index=[0],
    )

    return df_perf

In [ ]:
model_performance_classification_sklearn(grid, X_train, y_train)

In [ ]:
model_performance_classification_sklearn(grid, X_test, y_test)

In [ ]:
df1 = pd.DataFrame(model_performance_classification_sklearn(grid, X_test, y_test).T).reset_index().rename(columns={'index':'Metrics_names',0:'Testing_Metrics'})
df1

In [ ]:
df2 = pd.DataFrame(model_performance_classification_sklearn(grid, X_train, y_train).T).reset_index().rename(columns={'index':'Metrics_names',0:'Training_Metrics'})
df2

In [ ]:
Output_df = pd.merge(df2, df1, on='Metrics_names')
display(Output_df)

In [ ]:
def confusion_matrix_sklearn(model, predictors, target):
    """
    To plot the confusion_matrix with percentages

    model: classifier
    predictors: independent variables
    target: dependent variable
    """
    y_pred = model.predict(predictors)
    cm = confusion_matrix(target, y_pred)
    labels = np.asarray(
        [
            ["{0:0.0f}".format(item) + "\n{0:.2%}".format(item / cm.flatten().sum())]
            for item in cm.flatten()
        ]
    ).reshape(2, 2)

    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=labels, fmt="")
    plt.ylabel("True label")
    plt.xlabel("Predicted label")

In [ ]:
confusion_matrix_sklearn(grid, X_train, y_train)

In [ ]:
confusion_matrix_sklearn(grid, X_test, y_test)